In [ ]:
!pip install opencv-python

In [ ]:
!pip install mediapipe

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install seaborn

**Importing needed models**

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import plot_model

**Determine the dataset path**

In [ ]:
dataset = r'D:\Coaching Dataset'


directory = r"D:\Coatching"


os.makedirs(directory, exist_ok=True)


model_name = "model_name"


model_save_path = os.path.join(directory, model_name + ".keras")


**Determine the number of classes**

In [ ]:
NUM_CLASSES = 6

**Define the load date function**

In [ ]:
def load_dataset(dataset_path):
    x = []
    y = []
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    # Loop through dataset directory
    for exercise_folder in os.listdir(dataset_path):
        exercise_label = exercise_folder
        exercise_folder_path = os.path.join(dataset_path, exercise_folder)  #/content/Data1/crunches

        # Loop through video files in exercise folder
        for video_file in os.listdir(exercise_folder_path):
            video_path = os.path.join(exercise_folder_path, video_file)
            cap = cv2.VideoCapture(video_path)
            while cap.isOpened():
                success, image = cap.read()
                if not success:
                    break
                # Process image using MediaPipe Pose Detection
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                result = pose.process(image_rgb)
                if result.pose_landmarks:
                    # Extract pose landmarks
                    landmarks = [[lm.x, lm.y] for lm in result.pose_landmarks.landmark]
                    x.append(landmarks)
                    y.append(exercise_label)
            cap.release()
    return np.array(x), np.array(y)

**Call the load data function**

In [ ]:
x, y = load_dataset(dataset)


In [ ]:
print(f"Type of X: {type(x)}")
print(f"Type of y: {type(y)}")

**Change the label's names**

In [ ]:
for i in range(y.size):
  if y[i] == 'Chest Fly Machine False':
    y[i] = 0
  elif y[i] == 'Chest Fly Machine True':
    y[i] = 1
  elif y[i]=='Hammer Curl False':
    y[i]=2
  elif y[i]=='Hammer Curl True':
    y[i]=3
  elif y[i]=='Leg Extension False':
    y[i]=4
  elif y[i]=='Leg Extension True':
    y[i]=5
  else:
    y[i] = -1

**Making sure that the x and y is np.float32**

In [ ]:
if x.dtype != np.float32:
    x = x.astype(np.float32)
if y.dtype != np.float32:
    y = y.astype(np.float32)

**Divide the dataset into train data and test data**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

**Building the model**

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])


**Defining the call back function**

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

**Compile the model**

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

**Training the model**

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=150,
    batch_size=64,
    validation_split=0.2,
    callbacks=[cp_callback, es_callback]
    )


In [ ]:
model.summary()


**Evaluate data**

In [ ]:


loss, accuracy = model.evaluate(X_test, y_test)


**Load the model**

In [ ]:
model = tf.keras.models.load_model(model_save_path)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report 
import seaborn as sns 
import matplotlib.pyplot as plt 

# Predict labels for test data 
y_pred = np.argmax(model.predict(X_test), axis=1) 
# Error Analysis 
misclassifications = np.where(y_test != y_pred)[0] 
print("Misclassified Instances:", misclassifications) 

# Result Analysis 
accuracy = np.mean(y_test == y_pred) 
print("Accuracy:", accuracy) 
print(classification_report(y_test, y_pred)) 

# Confusion Matrix 
conf_mat = confusion_matrix(y_test, y_pred) 
plt.figure(figsize=(8, 6)) 
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues') 
plt.xlabel('Predicted Labels') 
plt.ylabel('True Labels') 
plt.title('Confusion Matrix') 
plt.show() 

# Visualization for Test and Train (Learning curves) 
plt.plot(history.history['accuracy'], label='accuracy') 
plt.plot(history.history['val_accuracy'], label = 'val_accuracy') 
plt.xlabel('Epoch') 
plt.ylabel('Accuracy') 
plt.ylim([0, 1]) 
plt.legend(loc='lower right') 
plt.show() 

plt.plot(history.history['loss'], label='loss') 
plt.plot(history.history['val_loss'], label = 'val_loss') 
plt.xlabel('Epoch') 
plt.ylabel('Loss') 
plt.ylim([0, 1]) 
plt.legend(loc='upper right') 
plt.show()

**Test the model by videos**

In [ ]:
video_path = r'C:\Users\User\Documents\Coaching Dataset\Chest Fly Machine Right_Wrong(240P).mp4'
cap = cv2.VideoCapture(video_path)

In [ ]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Process frame with MediaPipe Pose Detection
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(frame_rgb)

    # Check if pose landmarks are detected
    if result.pose_landmarks:
        # Extract pose landmarks
        landmarks = [[lm.x, lm.y] for lm in result.pose_landmarks.landmark]

        # Preprocess landmarks (reshape, convert to numpy array, etc.)
        # Example:
        landmarks_array = np.array(landmarks, dtype=np.float32)
        landmarks_array = landmarks_array[np.newaxis, ...]  # Add batch dimension

        # Use the model to make predictions
        predictions = model.predict(landmarks_array)

        # Map predictions to class labels
        class_labels = {
            0: "Chest Fly Machine False",
            1: "Chest Fly Machine True",
            2: "Hammer Curl False",
            3: "Hammer Curl True",
            4: "Leg Extension False",
            5: "Leg Extension True"
        }

        # Get the predicted class label text
        predicted_label = class_labels[np.argmax(predictions)]

        # Draw a colored background rectangle
        cv2.rectangle(frame, (0, 0), (frame.shape[1], 45), (225, 255, 218), -1)

        # Write the predicted label text on the colored background
        cv2.putText(frame, predicted_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow("output", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()